In [ ]:
# data scraping

In [ ]:
import time
import re
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


def parse_votes(votes_raw: str) -> int:
    """Convert '(45K)', '(1.2M)', '(987)' -> 45000 / 1200000 / 987.
       Handles thin/non-breaking spaces and stray chars."""
    if not votes_raw:
        return 0
    s = votes_raw.strip()                  # trim ends
    s = s.strip("()")                      # remove outer parens
    s = s.replace(",", "")                 # remove commas
    # remove various non-breaking/thin spaces
    s = s.replace("\u00A0", "").replace("\u202F", "").replace("\u2009", "")
    s = s.upper()
    # keep only digits, dot, and K/M suffix
    s = re.sub(r"[^0-9\.KM]", "", s)
    if not s:
        return 0
    mult = 1
    if s.endswith("K"):
        mult = 1000
        s = s[:-1]
    elif s.endswith("M"):
        mult = 1_000_000
        s = s[:-1]
    try:
        return int(float(s) * mult)
    except:
        return 0

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

genres = ["western"]

for genre in genres:
    all_movies = []
    url = f"https://www.imdb.com/search/title/?genres={genre}&release_date=2024-01-01,2024-12-31"
    driver.get(url)
    time.sleep(3)

    # keep clicking "50 more" until all movies load
    while True:
        try:
            more_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//button[.//span[text()="50 more"]]'))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(2)
        except TimeoutException:
            break

    cards = driver.find_elements(By.XPATH, '//li[@class="ipc-metadata-list-summary-item"]')

    for idx, card in enumerate(cards, 1):
        # Title
        try:
            title = card.find_element(By.XPATH, './/h3').text.strip()
            if ". " in title:   # remove "1. Title"
                title = title.split(". ", 1)[-1]
        except:
            title = ""

        # Rating
        try:
            rating = card.find_element(By.XPATH, './/span[@class="ipc-rating-star--rating"]').text.strip()
        except:
            rating = ""

        # Votes (robust)
        votes = 0
        try:
            # primary locator
            votes_raw = card.find_element(By.XPATH, './/span[contains(@class,"ipc-rating-star--voteCount")]').text
        except:
            # fallback: sometimes structure differs
            try:
                votes_raw = card.find_element(
                    By.XPATH,
                    './/div[contains(@class,"ipc-rating-star")]/following-sibling::span[contains(text(),"(")]'
                ).text
            except:
                votes_raw = ""
        votes = parse_votes(votes_raw)

        # Duration (regex)
        runtime = ""
        try:
            meta_items = card.find_elements(By.XPATH, './/span[contains(@class,"dli-title-metadata-item")]')
            for m in meta_items:
                txt = m.text.strip()
                if re.match(r"^\d+h\s*\d*m?$", txt) or re.match(r"^\d+h$", txt) or re.match(r"^\d+m$", txt):
                    runtime = txt
                    break
        except:
            pass

        all_movies.append([idx, title, rating, votes, runtime, genre])

    # Save each genre to its own CSV
    df = pd.DataFrame(all_movies, columns=["No", "Title", "Rating", "Votes", "Duration", "Genre"])
    df["Votes"] = df["Votes"].astype(int)  # ensur
    

out_path = os.path.join(os.getcwd(), f"{genre}_movies.csv")
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"✅ Saved {len(df)} movies to {out_path}")



In [4]:
import time
import re
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


def parse_votes(votes_raw: str) -> int:
    """Convert '(45K)', '(1.2M)', '(987)' -> 45000 / 1200000 / 987.
       Handles thin/non-breaking spaces and stray chars."""
    if not votes_raw:
        return 0
    s = votes_raw.strip()                  # trim ends
    s = s.strip("()")                      # remove outer parens
    s = s.replace(",", "")                 # remove commas
    # remove various non-breaking/thin spaces
    s = s.replace("\u00A0", "").replace("\u202F", "").replace("\u2009", "")
    s = s.upper()
    # keep only digits, dot, and K/M suffix
    s = re.sub(r"[^0-9\.KM]", "", s)
    if not s:
        return 0
    mult = 1
    if s.endswith("K"):
        mult = 1000
        s = s[:-1]
    elif s.endswith("M"):
        mult = 1_000_000
        s = s[:-1]
    try:
        return int(float(s) * mult)
    except:
        return 0

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

genres = ["war"]

for genre in genres:
    all_movies = []
    url = f"https://www.imdb.com/search/title/?genres={genre}&release_date=2024-01-01,2024-12-31"
    driver.get(url)
    time.sleep(3)

    # keep clicking "50 more" until all movies load
    while True:
        try:
            more_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//button[.//span[text()="50 more"]]'))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(2)
        except TimeoutException:
            break

    cards = driver.find_elements(By.XPATH, '//li[@class="ipc-metadata-list-summary-item"]')

    for idx, card in enumerate(cards, 1):
        # Title
        try:
            title = card.find_element(By.XPATH, './/h3').text.strip()
            if ". " in title:   # remove "1. Title"
                title = title.split(". ", 1)[-1]
        except:
            title = ""

        # Rating
        try:
            rating = card.find_element(By.XPATH, './/span[@class="ipc-rating-star--rating"]').text.strip()
        except:
            rating = ""

        # Votes (robust)
        votes = 0
        try:
            # primary locator
            votes_raw = card.find_element(By.XPATH, './/span[contains(@class,"ipc-rating-star--voteCount")]').text
        except:
            # fallback: sometimes structure differs
            try:
                votes_raw = card.find_element(
                    By.XPATH,
                    './/div[contains(@class,"ipc-rating-star")]/following-sibling::span[contains(text(),"(")]'
                ).text
            except:
                votes_raw = ""
        votes = parse_votes(votes_raw)

        # Duration (regex)
        runtime = ""
        try:
            meta_items = card.find_elements(By.XPATH, './/span[contains(@class,"dli-title-metadata-item")]')
            for m in meta_items:
                txt = m.text.strip()
                if re.match(r"^\d+h\s*\d*m?$", txt) or re.match(r"^\d+h$", txt) or re.match(r"^\d+m$", txt):
                    runtime = txt
                    break
        except:
            pass

        all_movies.append([idx, title, rating, votes, runtime, genre])

    # Save each genre to its own CSV
    df = pd.DataFrame(all_movies, columns=["No", "Title", "Rating", "Votes", "Duration", "Genre"])
    df["Votes"] = df["Votes"].astype(int)  # ensur
    

out_path = os.path.join(os.getcwd(), f"{genre}_movies.csv")
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"✅ Saved {len(df)} movies to {out_path}")



✅ Saved 2000 movies to c:\Users\priya\OneDrive\Desktop\Guvi coding\war_movies.csv


In [12]:
import time
import re
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


def parse_votes(votes_raw: str) -> int:
    """Convert '(45K)', '(1.2M)', '(987)' -> 45000 / 1200000 / 987.
       Handles thin/non-breaking spaces and stray chars."""
    if not votes_raw:
        return 0
    s = votes_raw.strip()                  # trim ends
    s = s.strip("()")                      # remove outer parens
    s = s.replace(",", "")                 # remove commas
    # remove various non-breaking/thin spaces
    s = s.replace("\u00A0", "").replace("\u202F", "").replace("\u2009", "")
    s = s.upper()
    # keep only digits, dot, and K/M suffix
    s = re.sub(r"[^0-9\.KM]", "", s)
    if not s:
        return 0
    mult = 1
    if s.endswith("K"):
        mult = 1000
        s = s[:-1]
    elif s.endswith("M"):
        mult = 1_000_000
        s = s[:-1]
    try:
        return int(float(s) * mult)
    except:
        return 0

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

genres = ["musical"]

for genre in genres:
    all_movies = []
    url = f"https://www.imdb.com/search/title/?genres={genre}&release_date=2024-01-01,2024-12-31"
    driver.get(url)
    time.sleep(3)

    # keep clicking "50 more" until all movies load
    while True:
        try:
            more_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//button[.//span[text()="50 more"]]'))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(2)
        except TimeoutException:
            break

    cards = driver.find_elements(By.XPATH, '//li[@class="ipc-metadata-list-summary-item"]')

    for idx, card in enumerate(cards, 1):
        # Title
        try:
            title = card.find_element(By.XPATH, './/h3').text.strip()
            if ". " in title:   # remove "1. Title"
                title = title.split(". ", 1)[-1]
        except:
            title = ""

        # Rating
        try:
            rating = card.find_element(By.XPATH, './/span[@class="ipc-rating-star--rating"]').text.strip()
        except:
            rating = ""

        # Votes (robust)
        votes = 0
        try:
            # primary locator
            votes_raw = card.find_element(By.XPATH, './/span[contains(@class,"ipc-rating-star--voteCount")]').text
        except:
            # fallback: sometimes structure differs
            try:
                votes_raw = card.find_element(
                    By.XPATH,
                    './/div[contains(@class,"ipc-rating-star")]/following-sibling::span[contains(text(),"(")]'
                ).text
            except:
                votes_raw = ""
        votes = parse_votes(votes_raw)

        # Duration (regex)
        runtime = ""
        try:
            meta_items = card.find_elements(By.XPATH, './/span[contains(@class,"dli-title-metadata-item")]')
            for m in meta_items:
                txt = m.text.strip()
                if re.match(r"^\d+h\s*\d*m?$", txt) or re.match(r"^\d+h$", txt) or re.match(r"^\d+m$", txt):
                    runtime = txt
                    break
        except:
            pass

        all_movies.append([idx, title, rating, votes, runtime, genre])

    # Save each genre to its own CSV
    df = pd.DataFrame(all_movies, columns=["No", "Title", "Rating", "Votes", "Duration", "Genre"])
    df["Votes"] = df["Votes"].astype(int)  # ensur
    

out_path = os.path.join(os.getcwd(), f"{genre}_movies.csv")
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"✅ Saved {len(df)} movies to {out_path}")



✅ Saved 3600 movies to c:\Users\priya\OneDrive\Desktop\Guvi coding\musical_movies.csv


In [18]:
import time
import re
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


def parse_votes(votes_raw: str) -> int:
    """Convert '(45K)', '(1.2M)', '(987)' -> 45000 / 1200000 / 987.
       Handles thin/non-breaking spaces and stray chars."""
    if not votes_raw:
        return 0
    s = votes_raw.strip()                  # trim ends
    s = s.strip("()")                      # remove outer parens
    s = s.replace(",", "")                 # remove commas
    # remove various non-breaking/thin spaces
    s = s.replace("\u00A0", "").replace("\u202F", "").replace("\u2009", "")
    s = s.upper()
    # keep only digits, dot, and K/M suffix
    s = re.sub(r"[^0-9\.KM]", "", s)
    if not s:
        return 0
    mult = 1
    if s.endswith("K"):
        mult = 1000
        s = s[:-1]
    elif s.endswith("M"):
        mult = 1_000_000
        s = s[:-1]
    try:
        return int(float(s) * mult)
    except:
        return 0

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

genres = ["biography"]

for genre in genres:
    all_movies = []
    url = f"https://www.imdb.com/search/title/?genres={genre}&release_date=2024-01-01,2024-12-31"
    driver.get(url)
    time.sleep(3)

    # keep clicking "50 more" until all movies load
    while True:
        try:
            more_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//button[.//span[text()="50 more"]]'))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(2)
        except TimeoutException:
            break

    cards = driver.find_elements(By.XPATH, '//li[@class="ipc-metadata-list-summary-item"]')

    for idx, card in enumerate(cards, 1):
        # Title
        try:
            title = card.find_element(By.XPATH, './/h3').text.strip()
            if ". " in title:   # remove "1. Title"
                title = title.split(". ", 1)[-1]
        except:
            title = ""

        # Rating
        try:
            rating = card.find_element(By.XPATH, './/span[@class="ipc-rating-star--rating"]').text.strip()
        except:
            rating = ""

        # Votes (robust)
        votes = 0
        try:
            # primary locator
            votes_raw = card.find_element(By.XPATH, './/span[contains(@class,"ipc-rating-star--voteCount")]').text
        except:
            # fallback: sometimes structure differs
            try:
                votes_raw = card.find_element(
                    By.XPATH,
                    './/div[contains(@class,"ipc-rating-star")]/following-sibling::span[contains(text(),"(")]'
                ).text
            except:
                votes_raw = ""
        votes = parse_votes(votes_raw)

        # Duration (regex)
        runtime = ""
        try:
            meta_items = card.find_elements(By.XPATH, './/span[contains(@class,"dli-title-metadata-item")]')
            for m in meta_items:
                txt = m.text.strip()
                if re.match(r"^\d+h\s*\d*m?$", txt) or re.match(r"^\d+h$", txt) or re.match(r"^\d+m$", txt):
                    runtime = txt
                    break
        except:
            pass

        all_movies.append([idx, title, rating, votes, runtime, genre])

    # Save each genre to its own CSV
    df = pd.DataFrame(all_movies, columns=["No", "Title", "Rating", "Votes", "Duration", "Genre"])
    df["Votes"] = df["Votes"].astype(int)  # ensur
    

out_path = os.path.join(os.getcwd(), f"{genre}_movies.csv")
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"✅ Saved {len(df)} movies to {out_path}")



✅ Saved 4800 movies to c:\Users\priya\OneDrive\Desktop\Guvi coding\biography_movies.csv


In [ ]:
# Consolidating the Scraped data

In [3]:
import pandas as pd
import os
import glob

# Folder where your individual genre CSVs are saved
csv_folder = os.getcwd()  # or specify the path
all_files = glob.glob(os.path.join(csv_folder, "*_movies.csv"))

# Read and concatenate all CSVs
df_list = [pd.read_csv(f) for f in all_files]
consolidated_df = pd.concat(df_list, ignore_index=True)

# Optional: reset numbering if your "No" column exists
if "No" in consolidated_df.columns:
    consolidated_df["No"] = range(1, len(consolidated_df)+1)

# Save consolidated CSV
consolidated_out = os.path.join(csv_folder, "all_genres_movies.csv")
consolidated_df.to_csv(consolidated_out, index=False, encoding="utf-8")

print(f"✅ Consolidated {len(consolidated_df)} movies into {consolidated_out}")


✅ Consolidated 22300 movies into c:\Users\priya\OneDrive\Desktop\Guvi coding\all_genres_movies.csv


In [ ]:
# Data Cleaning
# Data Type Conversion

In [4]:
# Path to your consolidated CSV
csv_path = os.path.join(os.getcwd(), "all_genres_movies.csv")
df = pd.read_csv(csv_path)

# Optional: convert columns to proper data types
df["Votes"] = pd.to_numeric(df["Votes"], errors='coerce').fillna(0).astype(int)
df["Rating"] = pd.to_numeric(df["Rating"], errors='coerce')
df["Title"] = df["Title"].astype(str)
df["Genre"] = df["Genre"].astype(str)
df["Duration"] = df["Duration"].astype(str)

In [5]:
# 🔹 Drop rows where BOTH Rating and Votes are missing (NaN or 0 for votes)
df = df.dropna(subset=["Rating", "Votes"], how="all")   # removes if both are NaN
df = df[~((df["Votes"] == 0) & (df["Rating"].isna()))]  # removes rows with 0 votes and no rating

# (Optional) If you also want to drop rows with Rating missing OR Votes = 0 separately:
# df = df[(df["Votes"] > 0) & (df["Rating"].notna())]

print(f"✅ Remaining movies after cleaning: {len(df)}")

✅ Remaining movies after cleaning: 5116


In [ ]:
display(df)

,No,Title,Rating,Votes,Duration,Genre
0,1,William Tell,5.7,4700,2h 14m,biography
1,2,A Complete Unknown,7.3,100000,2h 21m,biography
2,3,Saturday Night,6.9,35000,1h 49m,biography
3,4,I'm Still Here,8.2,118000,2h 17m,biography
4,5,Better Man,7.5,38000,2h 15m,biography
...,...,...,...,...,...,...
22222,22223,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.3,21,48m,western
22223,22224,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.2,20,53m,western
22224,22225,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.3,20,52m,western
22225,22226,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.8,21,53m,western


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5116 entries, 0 to 22226
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   No        5116 non-null   int64  
 1   Title     5116 non-null   object 
 2   Rating    5116 non-null   float64
 3   Votes     5116 non-null   int32  
 4   Duration  5116 non-null   object 
 5   Genre     5116 non-null   object 
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 259.8+ KB


In [9]:
# Data Cleaning 
# converting Duration into mins
import pandas as pd
import numpy as np
import re

# Load your CSV
df = pd.read_csv("all_genres_movies.csv")

def duration_to_minutes(d):
    if pd.isna(d) or d == "nan":
        return np.nan
    match = re.match(r"(?:(\d+)h)?\s*(?:(\d+)m)?", str(d))
    if match:
        h = int(match.group(1)) if match.group(1) else 0
        m = int(match.group(2)) if match.group(2) else 0
        return h*60 + m
    return np.nan

df["Duration_Min"] = df["Duration"].apply(duration_to_minutes)
display(df)


,No,Title,Rating,Votes,Duration,Genre,Duration_Min
0,1,William Tell,5.7,4700,2h 14m,biography,134.0
1,2,A Complete Unknown,7.3,100000,2h 21m,biography,141.0
2,3,Saturday Night,6.9,35000,1h 49m,biography,109.0
3,4,I'm Still Here,8.2,118000,2h 17m,biography,137.0
4,5,Better Man,7.5,38000,2h 15m,biography,135.0
...,...,...,...,...,...,...,...
22295,22296,Gegagedigedagedago,NaN,0,1m,western,1.0
22296,22297,Gegagedigedagedago,NaN,0,1m,western,1.0
22297,22298,Gegagedigedagedago,NaN,0,1m,western,1.0
22298,22299,The Forsaken Westerns,NaN,0,NaN,western,NaN


In [10]:
# Data Cleaning 
import pandas as pd
import numpy as np
import re

# Load your CSV
df = pd.read_csv("all_genres_movies.csv")

# --- Convert duration into minutes ---
def duration_to_minutes(d):
    if pd.isna(d) or d == "nan":
        return np.nan
    match = re.match(r"(?:(\d+)h)?\s*(?:(\d+)m)?", str(d))
    if match:
        h = int(match.group(1)) if match.group(1) else 0
        m = int(match.group(2)) if match.group(2) else 0
        return h*60 + m
    return np.nan

df["Duration_Min"] = df["Duration"].apply(duration_to_minutes)

# --- Convert numeric columns properly ---
df["Votes"] = pd.to_numeric(df["Votes"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# --- Drop movies with no rating AND no votes ---
df = df.dropna(subset=["Rating", "Votes"], how="all")       # both NaN
df = df[~((df["Votes"].fillna(0) == 0) & (df["Rating"].isna()))]  # 0 votes + no rating

# (Optional stricter cleaning: keep only rows with rating & votes)
# df = df[(df["Votes"] > 0) & (df["Rating"].notna())]

display(df)
print(f"✅ Cleaned dataset: {len(df)} movies left")


,No,Title,Rating,Votes,Duration,Genre,Duration_Min
0,1,William Tell,5.7,4700,2h 14m,biography,134.0
1,2,A Complete Unknown,7.3,100000,2h 21m,biography,141.0
2,3,Saturday Night,6.9,35000,1h 49m,biography,109.0
3,4,I'm Still Here,8.2,118000,2h 17m,biography,137.0
4,5,Better Man,7.5,38000,2h 15m,biography,135.0
...,...,...,...,...,...,...,...
22222,22223,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.3,21,48m,western,48.0
22223,22224,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.2,20,53m,western,53.0
22224,22225,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.3,20,52m,western,52.0
22225,22226,Golden Kamuy: The Hunt of Prisoners in Hokkaido,7.8,21,53m,western,53.0


✅ Cleaned dataset: 5116 movies left


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5116 entries, 0 to 22226
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   No            5116 non-null   int64  
 1   Title         5116 non-null   object 
 2   Rating        5116 non-null   float64
 3   Votes         5116 non-null   int64  
 4   Duration      2478 non-null   object 
 5   Genre         5116 non-null   object 
 6   Duration_Min  2478 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 319.8+ KB


In [12]:
# Saving cleaned data
df.to_csv("movies_cleaned.csv", index=False, encoding="utf-8")


In [ ]:
# DB Connection Establishement

In [26]:
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\priya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
# connection with cloud sql server
import mysql.connector
mydb = mysql.connector.connect(
 host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
 port = 4000,
 user="2DnwFtxWENyGXLF.root",
 password="3er4ekP3FXgFptI3",
)
print(mydb)
mycursor = mydb.cursor(buffered=True)


In [ ]:
# Creating DB and Table

In [20]:
# create database if not exists
mycursor.execute("CREATE DATABASE IF NOT EXISTS imdb;")

# select database
mycursor.execute("USE imdb;")


In [ ]:
create_table = """
CREATE TABLE IF NOT EXISTS movies (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Title VARCHAR(255),
    Rating FLOAT,
    Votes INT,
    Duration VARCHAR(50),
    Duration_min INT,
    Genre VARCHAR(50)
);
"""
mycursor.execute(create_table)
mydb.commit()


In [ ]:
# Inserting the Data in DB

In [23]:
mycursor.execute("SELECT Genre, COUNT(*) FROM movies GROUP BY Genre;")
for genre, count in mycursor.fetchall():
    print(genre, count)

biography 2464
war 1116
western 396
musical 1140


In [27]:
df_filtered = df[df["Votes"] > 0]

In [ ]:
@st.cache_data
def load_data():
    mydb = mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        port=4000,
        user="2DnwFtxWENyGXLF.root",
        password="3er4ekP3FXgFptI3",
        database="imdb"
    )
    query = "SELECT * FROM movies;"
    df = pd.read_sql(query, mydb)
    mydb.close()

    # Standardize column names
    df.columns = [col.strip().capitalize() for col in df.columns]

    # Ensure numeric conversion (if they exist)
    if "Rating" in df.columns:
        df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
    if "Votes" in df.columns:
        df["Votes"] = pd.to_numeric(df["Votes"], errors="coerce")
    if "Duration_min" in df.columns:
        df["Duration_min"] = pd.to_numeric(df["Duration_min"], errors="coerce")

    return df


In [ ]:
!pip install streamlit matplotlib seaborn plotly


In [ ]:
!pip install mysql-connector-python sqlalchemy pandas streamlit


In [ ]:
!pip install streamlit